In [1]:
%matplotlib inline

In [2]:
import LFPy
import lfpykit
import numpy as np

In [3]:
# LFPy.Cell parameters
cellParameters = {
    'morphology': 'L5_Mainen96_LFPy.hoc',  # morphology file
    'v_init': -65,                         # initial voltage
    'cm': 1.0,                             # membrane capacitance
    'Ra': 150,                             # axial resistivity
    'passive': True,                       # insert passive channels
    'passive_parameters': {"g_pas": 1. / 3E4,
                           "e_pas": -65},  # passive params
    'dt': 2**-4,                           # simulation time res
    'nsegs_method': 'lambda_f',    # discretization rule
    'lambda_f': 1000               # frequency (Hz)
}

# create LFPy.Cell instance
cell = LFPy.Cell(**cellParameters)
cell.set_rotation(x=4.98919, y=-4.33261, z=0.)

print(f'total number of segments: {cell.totnsegs}')

# parameters for line source potential
el_params = dict(
    x = np.linspace(0, 1000, 1001),
    y = np.zeros(1001),
    z = np.zeros(1001),
    sigma = 0.3
)

cell.simulate(rec_imem=True)

# create line-source potential predictor
lsp = lfpykit.LineSourcePotential(cell, **el_params)

total number of segments: 3078


In [4]:
%%prun -s cumulative -q -l 20 -T prun0
for i in range(100):
    lsp.get_transformation_matrix()
print(open('prun0', 'r').read())

         300548 function calls in 6.441 seconds

   Ordered by: cumulative time
   List reduced from 31 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    6.441    6.441 {built-in method builtins.exec}
        1    0.010    0.010    6.440    6.440 <string>:1(<module>)
      100    0.219    0.002    6.431    0.064 models.py:437(get_transformation_matrix)
   100100    0.277    0.000    6.211    0.000 lfpcalc.py:346(calc_lfp_linesource)
   100100    5.921    0.000    5.934    0.000 lfpcalc.py:377(_calc_lfp_linesource)
   100100    0.013    0.000    0.013    0.000 serialize.py:29(_numba_unpickle)
        1    0.000    0.000    0.000    0.000 {built-in method io.open}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        2    0.000    0.000    0.000    0.000 iostream.py:502(write)
        3    0.000    0.000    0.000    0.000 iostream.py:208(schedule)
        3    0.000    0.0

In [5]:
M = lsp.get_transformation_matrix()
M.shape

(1001, 3078)

In [6]:
cell.imem.shape

(3078, 1601)